In [ ]:
# Load the "autoreload" extension. Prior to executing code, modules are reloaded. 
# There's no need to restart jupyter notebook if you modify code in the `src` directory.
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

from bike_image_classifier_tensorflow.data.extract_bike_data import *
import pandas as pd
import requests
import urllib
import ast
from random import randint
from time import sleep
import pickle
import urllib.request
import urllib.error
import os

In [ ]:

# Extract bike lists from BBB API

bicycle_blue_book_search = 'https://api.bicyclebluebook.com/core/api/bicycles/brand/year/model?yearId={year}&page={page}'
extracted_bbb_2018 = extract_bbb_data(2018, 1, 2, bicycle_blue_book_search)

In [ ]:
# Create bike type folders

converted_type_set = set()
for bike_type in list(extracted_bbb_2018.typeName.unique()):
    converted_type_set.add(extract_data.bike_type_converter(bike_type))
    
for bike_type_directory in list(converted_type_set):
    path = '../data/raw/{}'
    os.mkdir(path.format(bike_type_directory))

In [ ]:
# Extract images

iterate_df = extracted_bbb_2018[['bicycleId','typeName','imageDefault']]

for i, row in enumerate(iterate_df.itertuples()):
    if pd.isna(row.imageDefault):
        print(i, 'Skipping')
        continue
    path = '../data/raw/{}/{}.png'
    adjusted_type=bike_type_converter(row.typeName)
    structured_path = path.format(adjusted_type, row.bicycleId)
    try:
        urllib.request.urlretrieve(row.imageDefault, structured_path)
    except urllib.error.HTTPError:
        print(i, 'Access Denied')
    print(i, structured_path)